## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_value_counts = application_df['APPLICATION_TYPE'].value_counts()
application_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 30
application_types_to_replace = application_value_counts[application_value_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1370        1
C1570        1
C1248        1
C1900        1
C2500        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts[classification_value_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
C1600        5
C1257        5
C0           3
C1260        3
C2710        3
C1267        2
C1256        2
C1246        2
C1234        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100
classifications_to_replace = classification_value_counts[classification_value_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop('IS_SUCCESSFUL', axis=1)
y = dummies_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
X_train.shape[1]

51

In [14]:
from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger('model_results.csv', append=True)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  52
hidden_nodes_layer2 = 16

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 52)                2704      
                                                                 
 dense_13 (Dense)            (None, 16)                848       
                                                                 
 dense_14 (Dense)            (None, 1)                 17        
                                                                 
Total params: 3,569
Trainable params: 3,569
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
class CustomCSVLogger(CSVLogger):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:
            super().on_epoch_end(epoch, logs)

In [28]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=CustomCSVLogger('model_results.csv', append=True))

Epoch 1/100
804/804 [==============================] - 1s 780us/step - loss: 0.5766 - accuracy: 0.7157
Epoch 2/100
804/804 [==============================] - 1s 758us/step - loss: 0.5537 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 1s 787us/step - loss: 0.5496 - accuracy: 0.7325
Epoch 4/100
804/804 [==============================] - 1s 742us/step - loss: 0.5489 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 1s 723us/step - loss: 0.5472 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 1s 753us/step - loss: 0.5462 - accuracy: 0.7337
Epoch 7/100
804/804 [==============================] - 1s 739us/step - loss: 0.5454 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 1s 727us/step - loss: 0.5445 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 1s 757us/step - loss: 0.5436 - accuracy: 0.7356
Epoch 10/100
804/804 [==============================] - 1s 729us/step - l

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5531 - accuracy: 0.7257 - 222ms/epoch - 828us/step
Loss: 0.5531392097473145, Accuracy: 0.7257142663002014


## NN Model Optimisation

Optimisation with original dataset

In [30]:
# Tuning Parameters
activation_functions = ['relu', 'sigmoid', 'tanh']
hidden_nodes_layer1_values = [32, 64, 128]
hidden_nodes_layer2_values = [16, 32, 64]
optimizers = ['adam', 'rmsprop']
losses = ['binary_crossentropy', 'mean_squared_error']

In [40]:
parameter_combinations = list(itertools.product(activation_functions, hidden_nodes_layer1_values, hidden_nodes_layer2_values, optimizers, losses))
parameter_combinations[103]

('tanh', 128, 32, 'rmsprop', 'mean_squared_error')

In [32]:
def train_model(activation, hidden_nodes_layer1, hidden_nodes_layer2, optimizer, loss):
    nn = tf.keras.models.Sequential()
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation))
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation))
    nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    nn.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    nn.fit(X_train_scaled, y_train, epochs=20, callbacks=[CustomCSVLogger('model_tuning_results.csv', append=True)])

In [34]:
results_df = pd.DataFrame(columns=['Activation', 'Hidden Nodes Layer 1', 'Hidden Nodes Layer 2', 'Optimizer', 'Loss'])
for params in parameter_combinations:
    activation, hidden_nodes_layer1, hidden_nodes_layer2, optimizer, loss = params
    train_model(activation, hidden_nodes_layer1, hidden_nodes_layer2, optimizer, loss)
    results_df = results_df.append({'Activation': activation,
                                    'Hidden Nodes Layer 1': hidden_nodes_layer1,
                                    'Hidden Nodes Layer 2': hidden_nodes_layer2,
                                    'Optimizer': optimizer,
                                    'Loss': loss}, ignore_index=True)
    
results_df.to_csv('model_parameters.csv', index=False)

Epoch 1/20
804/804 [==============================] - 1s 842us/step - loss: 0.5793 - accuracy: 0.7150
Epoch 2/20
804/804 [==============================] - 1s 740us/step - loss: 0.5522 - accuracy: 0.7310
Epoch 3/20
804/804 [==============================] - 1s 813us/step - loss: 0.5496 - accuracy: 0.7324
Epoch 4/20
804/804 [==============================] - 1s 766us/step - loss: 0.5477 - accuracy: 0.7326
Epoch 5/20
804/804 [==============================] - 1s 722us/step - loss: 0.5469 - accuracy: 0.7335
Epoch 6/20
804/804 [==============================] - 1s 731us/step - loss: 0.5459 - accuracy: 0.7338
Epoch 7/20
804/804 [==============================] - 1s 737us/step - loss: 0.5450 - accuracy: 0.7344
Epoch 8/20
804/804 [==============================] - 1s 705us/step - loss: 0.5446 - accuracy: 0.7342
Epoch 9/20
804/804 [==============================] - 1s 704us/step - loss: 0.5444 - accuracy: 0.7345
Epoch 10/20
804/804 [==============================] - 1s 708us/step - loss: 0.543

In [35]:
# Removed organisation and special consideration
dummies_reduced_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE'])
dummies_reduced_df

,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,...,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,Association,1,0,N,5000,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Co-operative,1,1-9999,N,108590,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Association,1,0,N,5000,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,Trust,1,10000-24999,N,6692,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Trust,1,100000-499999,N,142590,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,Association,1,0,N,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34295,Association,1,0,N,5000,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
34296,Association,1,0,N,5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,Association,1,0,N,5000,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [36]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop('IS_SUCCESSFUL', axis=1)
y = dummies_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train.shape[1]

51

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  128
hidden_nodes_layer2 = 32

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_115"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_346 (Dense)           (None, 128)               6656      
                                                                 
 dense_347 (Dense)           (None, 32)                4128      
                                                                 
 dense_348 (Dense)           (None, 1)                 33        
                                                                 
Total params: 10,817
Trainable params: 10,817
Non-trainable params: 0
_________________________________________________________________


In [44]:
nn.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [47]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=200, callbacks=CustomCSVLogger('model_refined_results.csv', append=True))

Epoch 1/200
804/804 [==============================] - 1s 947us/step - loss: 0.5340 - accuracy: 0.7416
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5339 - accuracy: 0.7432
Epoch 3/200
804/804 [==============================] - 1s 924us/step - loss: 0.5341 - accuracy: 0.7417
Epoch 4/200
804/804 [==============================] - 1s 899us/step - loss: 0.5339 - accuracy: 0.7427
Epoch 5/200
804/804 [==============================] - 1s 951us/step - loss: 0.5337 - accuracy: 0.7423
Epoch 6/200
804/804 [==============================] - 1s 939us/step - loss: 0.5338 - accuracy: 0.7425
Epoch 7/200
804/804 [==============================] - 1s 888us/step - loss: 0.5341 - accuracy: 0.7425
Epoch 8/200
804/804 [==============================] - 1s 847us/step - loss: 0.5338 - accuracy: 0.7426
Epoch 9/200
804/804 [==============================] - 1s 919us/step - loss: 0.5342 - accuracy: 0.7428
Epoch 10/200
804/804 [==============================] - 1s 913us/step - los

In [46]:
# Export our model to HDF5 file

nn.save("AlphabetSoupCharity_Optimisation.h5")